## 相当于是一个阅读笔记吧，会将[wtfpython: A collection of surprising Python snippets and lesser-known features](https://github.com/satwikkansal/wtfpython) (中文版: [中文](https://github.com/leisurelicht/wtfpython-cn))中遇到一些我认为值得思考的代码片段记下来。

## Section: Strain your brain!

### >  Return return everywhere!

In [1]:
def some_func():
    try:
        return 'from_try'
    finally:
        return 'from_finally'

In [2]:
some_func()

'from_finally'

<hr>
💡
* <font color='red'>"try...finally" 语句的 try 中执行 return, break 或 continue 后, finally 子句依然会执行.</font>
* 函数的返回值由最后执行的 return 语句决定. 所以 finally 子句中的 return 将是最后执行的语句.

也就是说，尽量避免在try语句中写return语句(毕竟try 语句仅用来处理有可能出错的代码).

另外，一些语言(如C#)会禁止你在finally 语句中 return.

### > For what?

In [3]:
some_string = "wtf"
some_dict = {}
for i, some_dict[i] in enumerate(some_string):
    pass

In [4]:
some_dict

{0: 'w', 1: 't', 2: 'f'}

<hr>
💡
Python 语法 中对 for 的定义是:`for_stmt: 'for' exprlist 'in' testlist ':' suite ['else' ':' suite]`

其中 exprlist 指分配目标. 这意味着对可迭代对象中的每一项都会执行类似 {exprlist} = {next_value} 的操作. 在上例中，每次循环都有some_dict[i] = enumerate(some_string)产生的字符串, 即 some_dict[0] = 'w', ...... 

一个例子：该循环只循环一次？

In [5]:
for i in range(4):
    print(i)
    i = 10

0
1
2
3


### > Evaluation time discrepancy/执行时机差异

In [6]:
array = [1, 8, 15]
g = (x for x in array if array.count(x) == 1)
array = [2, 8, 22]

list(g)

[8]

In [7]:
array_1 = [1,2,3,4]
g1 = (x for x in array_1)
array_1 = [1,2,3,4,5]

array_2 = [1,2,3,4]
g2 = (x for x in array_2)
array_2[:] = [1,2,3,4,5]

print(list(g1))
print(list(g2))

[1, 2, 3, 4]
[1, 2, 3, 4, 5]


💡 

* 在生成器表达式中, in 子句在声明时执行, 而条件子句则是在运行时执行.
* 所以在运行前, array 已经被重新赋值为 [2, 8, 22], 因此对于之前的 1, 8 和 15, 只有 count(8) 的结果是等于 1 的, 所以生成器只会生成 8. <font color='red'>???</font>
* 第二部分中 g1 和 g2 的输出差异则是由于变量 array_1 和 array_2 被重新赋值的方式导致的.
* 在第一种情况下, array_1 被绑定到新对象 [1,2,3,4,5], 因为 in 子句是在声明时被执行的， 所以它仍然引用旧对象 [1,2,3,4](并没有被销毁).
* 在第二种情况下, 对 array_2 的切片赋值将相同的旧对象 [1,2,3,4] 原地更新为 [1,2,3,4,5]. 因此 g2 和 array_2 仍然引用同一个对象(这个对象现在已经更新为 [1,2,3,4,5]).

### > What's wrong with booleans?

In [8]:
# 一个简单的例子, 统计下面可迭代对象中的布尔型值的个数和整型值的个数
mixed_list = [False, 1.0, "some_string", 3, True, [], False]
integers_found_so_far = 0
booleans_found_so_far = 0

for item in mixed_list:
    if isinstance(item, int):
        integers_found_so_far += 1
    elif isinstance(item, bool):
        booleans_found_so_far += 1

In [9]:
integers_found_so_far, booleans_found_so_far

(4, 0)

In [10]:
another_dict = {}
another_dict[True] = "JavaScript"
another_dict[1] = "Ruby"
another_dict[1.0] = "Python"

In [11]:
another_dict[True]

'Python'

In [12]:
>>> some_bool = True
>>> "wtf"*some_bool
'wtf'
>>> some_bool = False
>>> "wtf"*some_bool
''

''

💡
* 布尔值是 int 的子类
* 所以 True 的整数值是 1, 而 False 的整数值是 0.

### > Mutating the immutable

In [13]:
some_tuple = ("A", "tuple", "with", "values")
another_tuple = ([1, 2], [3, 4], [5, 6])

In [14]:
some_tuple[2] = "change this"

TypeError: 'tuple' object does not support item assignment

In [15]:
another_tuple[2].append(1000) # 这里不出现错误

In [16]:
another_tuple

([1, 2], [3, 4], [5, 6, 1000])

In [17]:
another_tuple[2] += [99, 999]

TypeError: 'tuple' object does not support item assignment

In [18]:
another_tuple

([1, 2], [3, 4], [5, 6, 1000, 99, 999])

💡 
* 不可变序列 不可变序列的对象一旦创建就不能再改变. (如果对象包含对其他对象的引用，则这些其他对象可能是可变的并且可能会被修改; 但是，由不可变对象直接引用的对象集合不能更改.)
* += 操作符在原地修改了列表. 元素赋值操作并不工作, 但是当异常抛出时, 元素已经在原地被修改了.

补充: 对于+=这个运算符,如a+=b:

* 对于可变对象(mutable object)如list, +=操作的结果会直接在a对应的变量进行修改，而a对应的地址不变.
* 对于不可变对象(imutable object)如tuple, +=则是等价于a = a+b 会产生新的变量，然后绑定到a上而已.

对于tuple:
* tuple内部的元素不支持赋值操作
* 在第一条的基础上, 如果元素的id没有变化, 元素其实是可以改变的.

详细解释可以看[这里](https://segmentfault.com/a/1190000010767068)

## Section: Appearances are deceptive!

In [19]:
value = 11
valuе = 32
value

11

💡
一些非西方字符虽然看起来和英语字母相同, 但会被解释器识别为不同的字母. 🤡

In [20]:
ord('е') # 西里尔语的 'e' (Ye)

1077

In [21]:
ord('e') # ascii的 'e', 用于英文并使用标准键盘输入

101

In [22]:
import numpy as np

def energy_send(x):
    # 初始化一个 numpy 数组
    np.array([float(x)])

def energy_receive():
    # 返回一个空的 numpy 数组
    return np.empty((), dtype=np.float).tolist()

In [23]:
energy_send(123.456)
energy_receive()

123.456

💡
注意在 energy_send 函数中创建的 numpy 数组并没有返回, 因此内存空间被释放并可以被重新分配.
numpy.empty() 直接返回下一段空闲内存，而不重新初始化. 而这个内存点恰好就是刚刚释放的那个(通常情况下, 并不绝对).

## Section: Watch out for the landmines!

### > Stubborn del operato

In [24]:
# 24 ~ 26 在ipython kernel下不能实现
class SomeClass:
    def __del__(self):
        print("Deleted!")

x = SomeClass()
y = x
del x
y # 检查一下y是否存在

In [25]:
del y # 这里应该会输出 "Deleted!"
# 执行 globals 然后会打印 "Deleted"

💡
* del x 并不会立刻调用 x.__del__().
* 每当遇到 del x, Python 会将 x 的引用数减1, 当 x 的引用数减到0时就会调用 x.__del__().
* 调用 globals 导致引用被销毁, 因此我们可以看到 "Deleted!" 终于被输出了.

In [26]:
list_1, list_2 = [1, 2, 3, 4],  [1, 2, 3, 4]

for idx, item in enumerate(list_1[:]):
    list_1.remove(item)

for idx, item in enumerate(list_2):
    print('idx:', idx, 'list:', list_2)
    list_2.pop(idx)

idx: 0 list: [1, 2, 3, 4]
idx: 1 list: [2, 3, 4]


In [27]:
list_1

[]

In [28]:
list_2

[2, 4]

💡
* 在迭代时修改对象是一个很愚蠢的主意. 正确的做法是迭代对象的副本, list_1[:] 就是这么做的.

### > Loop variables leaking out!

In [29]:
x = -1
for x in range(7):
    if x == 6:
        print(x, ': for x inside loop')
print(x, ': x in global')

6 : for x inside loop
6 : x in global


In [30]:
x = -1
print([x for x in range(5)])
print(x, ': x in global')

[0, 1, 2, 3, 4]
-1 : x in global


💡
* 在 Python 中, for 循环使用所在作用域并在结束后保留定义的循环变量. 如果我们曾在全局命名空间中定义过循环变量. 在这种情况下, 它会重新绑定现有变量.
* 另外, 注意列表推导具有不同的语义: 它们更接近于 list() 构造函数中生成器表达式的语法躺.

### > Beware of default mutable arguments!

In [1]:
def foo(color=['red']):
    color.append('blue')
    return color

In [2]:
foo()

['red', 'blue']

In [3]:
foo()

['red', 'blue', 'blue']

💡
* Python中函数的默认<font color='red'>可变</font>参数并不是每次调用该函数时都会被初始化. 相反, 它们会使用最近分配的值作为默认值. 而当我们明确的将 [] 作为参数传递给 foo 的时候, 就不会使用 color 的默认值.
* 避免可变参数导致的错误的常见做法是将 None 指定为参数的默认值, 然后检查是否有值传给对应的参数.

### > Be careful with chained operations!

In [4]:
False == (False in [False])

False

In [5]:
# 注意!
False == False in [False]

True

In [6]:
False is False is False

True

In [7]:
1 > 0 < 1

True

In [8]:
(1 > 0) < 1

False

💡想想以下的结果是什么：
* `0 <= x <= 100`
* 另外：`int(False)` 的结果为 0

### > Name resolution ignoring class scope/忽略类作用域的名称解析

In [9]:
x = 5
class SomeClass:
    x = 17
    y = (x for i in range(10))

In [10]:
list(SomeClass.y)[0]

5

In [11]:
x = 5
class SomeClass:
    x = 17
    y = [x for i in range(10)]

In [12]:
SomeClass.y[0]

5

💡
* 类定义中嵌套的作用域会忽略类内的名称绑定.
* 生成器表达式有它自己的作用域.
* 列表推导式也有自己的作用域.